In [ ]:
import geopandas as gpd

# 1. 정류소 데이터 로드 (GIS 파일 로드)
crosswalk_gdf = gpd.read_file("./횡단보도_원본데이터/부산광역시 신호등 설치 횡단보도 위치정보.zip",encoding="cp949")  # 정류소 데이터 GIS 파일 로드

# 2. 정류소 데이터의 좌표계를 EPSG:4326으로 설정 (WGS84)
crosswalk_gdf = crosswalk_gdf.set_crs(epsg=4326)

# 3. 행정구역 경계 데이터 로드
admin_boundary_dong = gpd.read_file("./횡단보도_원본데이터/BND_ADM_DONG_PG.zip",encoding="cp949")  # 부산시 행정구역 경계 파일 로드

# 4. 좌표계 통일 (WGS84로 변환)
admin_boundary_dong = admin_boundary_dong.to_crs(epsg=4326)

# 5. 횡당보도와 행정동 간 공간 조인
# 'within' 대신 'intersects' 또는 'contains'를 사용할 수 있습니다.
crosswalk_gdf_with_dong = gpd.sjoin(crosswalk_gdf, admin_boundary_dong, how="left", predicate='intersects')

# 6. 결과 저장 (구별로 분류된 데이터 저장)
crosswalk_gdf_with_dong[['address_do','geometry', 'ADM_NM']].to_csv("crosswalk_by_dong.csv", index=False, encoding='cp949')

print("동별로 분류된 데이터가 저장되었습니다.")

# 7. 동별 정류소 개수 집계
crosswalk_counts = crosswalk_gdf_with_dong.groupby('ADM_NM').size().reset_index(name='횡단보도 개수')

# 8. 결과 저장
crosswalk_counts.to_csv("crosswalk_counts_by_dong.csv", index=False, encoding='cp949')

print("동별 횡단보도 개수가 저장되었습니다.")